In [1]:
# ================= CELL EVALUASI: MODEL LEADERBOARD =================
from sentence_transformers import SentenceTransformer, util
from scipy.stats import spearmanr
import pandas as pd
import os
import torch
from tqdm.notebook import tqdm

# --- 1. KONFIGURASI PATH ---
# Masukkan path folder tempat kamu menyimpan HASIL TRAINING (bukan model mentah)
TRAINED_MODELS = {
    "Indo-E5 Small": r"D:\71210730\Eval\\\Trained_SBERT\\finetuned_indo_e5_v1",
    "SimCSE IndoBERT": r"D:\71210730\Eval\\\Trained_SBERT\\finetuned_simcse-indobert-base_v1", 
    "NusaBERT Base": r"D:\71210730\Eval\\\Trained_SBERT\\finetuned_all-nusabert-base_v1",
    "NusaBERT Large": r"D:\71210730\Eval\\\Trained_SBERT\\finetuned_nusabert_large_v1"
}

TEST_DATA_PATH = "D:\\71210730\\Versi 4\\Training SBERT\\test_sbert.csv"

# --- 2. LOAD TEST DATA ---
print(f"Membaca data uji: {TEST_DATA_PATH}")
df_test = pd.read_csv(TEST_DATA_PATH, encoding='utf-8-sig')
print(f"Jumlah Data Uji: {len(df_test)}")

# Ambil list kalimat dan skor label
sentences1 = df_test['text_1'].tolist()
sentences2 = df_test['text_2'].tolist()
gold_scores = df_test['score_norm'].tolist()

# --- 3. FUNGSI EVALUASI ---
def evaluate_model(model_path, model_name):
    print(f"\nSedang menguji: {model_name}...")
    
    # Cek apakah folder ada
    if not os.path.exists(model_path):
        print(f"❌ Path tidak ditemukan: {model_path}")
        return None

    try:
        # Load Model
        model = SentenceTransformer(model_path)
        
        # Cek apakah butuh prefix (khusus E5)
        # Kita pakai logika sederhana: kalau namanya mengandung 'e5', tambah query
        prefix = "query: " if "e5" in model_name.lower() else ""
        
        # Encode (Batching biar aman memori)
        # Tambahkan prefix jika perlu
        input_s1 = [prefix + str(s) for s in sentences1]
        input_s2 = [prefix + str(s) for s in sentences2]
        
        emb1 = model.encode(input_s1, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
        emb2 = model.encode(input_s2, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
        
        # Hitung Cosine Similarity
        cosine_scores = util.cos_sim(emb1, emb2).diagonal().cpu().numpy()
        
        # Hitung Spearman Correlation
        corr, _ = spearmanr(gold_scores, cosine_scores)
        print(f"✅ Selesai. Spearman: {corr:.4f}")
        
        return {
            "Nama Model": model_name,
            "Spearman Correlation": corr,
            "Path": model_path
        }
        
    except Exception as e:
        print(f"❌ Error pada {model_name}: {e}")
        return None

# --- 4. EKSEKUSI LOOP ---
results = []

for name, path in TRAINED_MODELS.items():
    res = evaluate_model(path, name)
    if res:
        results.append(res)

# --- 5. TAMPILKAN LEADERBOARD ---
print("\n" + "="*40)
print("🏆 HASIL AKHIR EVALUASI (LEADERBOARD) 🏆")
print("="*40)

if results:
    df_results = pd.DataFrame(results)
    # Urutkan dari skor tertinggi
    df_results = df_results.sort_values(by="Spearman Correlation", ascending=False).reset_index(drop=True)
    
    print(df_results[["Nama Model", "Spearman Correlation"]])
    
    # Simpan ke CSV untuk laporan skripsi
    df_results.to_csv("hasil_evaluasi_model.csv", index=False)
    print("\nFile tersimpan: hasil_evaluasi_model.csv")
    
    best_model_name = df_results.iloc[0]['Nama Model']
    print(f"\n🥇 Juara 1: {best_model_name}")
    print("Gunakan model ini untuk sistem Backend ScripTi kamu!")
else:
    print("Tidak ada hasil evaluasi.")

c:\Python\Python3.11.9\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Membaca data uji: D:\71210730\Versi 4\Training SBERT\test_sbert.csv
Jumlah Data Uji: 6000

Sedang menguji: Indo-E5 Small...


Batches: 100%|██████████| 94/94 [00:06<00:00, 15.16it/s]


✅ Selesai. Spearman: 0.8894

Sedang menguji: SimCSE IndoBERT...


Batches: 100%|██████████| 94/94 [00:21<00:00,  4.39it/s]


✅ Selesai. Spearman: 0.8958

Sedang menguji: NusaBERT Base...


Batches: 100%|██████████| 94/94 [00:24<00:00,  3.85it/s]


✅ Selesai. Spearman: 0.8983

Sedang menguji: NusaBERT Large...


Batches: 100%|██████████| 94/94 [01:10<00:00,  1.34it/s]


✅ Selesai. Spearman: 0.8796

🏆 HASIL AKHIR EVALUASI (LEADERBOARD) 🏆
        Nama Model  Spearman Correlation
0    NusaBERT Base              0.898317
1  SimCSE IndoBERT              0.895776
2    Indo-E5 Small              0.889431
3   NusaBERT Large              0.879556

File tersimpan: hasil_evaluasi_model.csv

🥇 Juara 1: NusaBERT Base
Gunakan model ini untuk sistem Backend ScripTi kamu!


In [1]:
# ================= CELL EVALUASI LENGKAP (SPEARMAN + PEARSON) =================
from sentence_transformers import SentenceTransformer, util
from scipy.stats import spearmanr, pearsonr
import pandas as pd
import os
import torch

# --- KONFIGURASI ---
# Gunakan path yang sama seperti tadi
TRAINED_MODELS = {
    "Indo-E5 Small": r"D:\71210730\Eval\\\Trained_SBERT\\finetuned_indo_e5_v1",
    "SimCSE IndoBERT": r"D:\71210730\Eval\\\Trained_SBERT\\finetuned_simcse-indobert-base_v1", 
    "NusaBERT Base": r"D:\71210730\Eval\\\Trained_SBERT\\finetuned_all-nusabert-base_v1",
    "NusaBERT Large": r"D:\71210730\Eval\\\Trained_SBERT\\finetuned_nusabert_large_v1"
}

TEST_DATA_PATH = "D:\\71210730\\Versi 4\\Training SBERT\\test_sbert.csv"

# --- LOAD DATA ---
df_test = pd.read_csv(TEST_DATA_PATH, encoding='utf-8-sig')
sentences1 = df_test['text_1'].tolist()
sentences2 = df_test['text_2'].tolist()
gold_scores = df_test['score_norm'].tolist()

results = []

print("Mulai Evaluasi Komprehensif...")

for model_name, model_path in TRAINED_MODELS.items():
    print(f"\n🔹 Menguji: {model_name}...")
    try:
        model = SentenceTransformer(model_path)
        
        prefix = "query: " if "e5" in model_name.lower() else ""
        input_s1 = [prefix + str(s) for s in sentences1]
        input_s2 = [prefix + str(s) for s in sentences2]
        
        emb1 = model.encode(input_s1, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
        emb2 = model.encode(input_s2, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
        
        cosine_scores = util.cos_sim(emb1, emb2).diagonal().cpu().numpy()
        
        # Hitung Spearman
        spearman_score, _ = spearmanr(gold_scores, cosine_scores)
        
        # Hitung Pearson (TAMBAHAN)
        pearson_score, _ = pearsonr(gold_scores, cosine_scores)
        
        print(f"   >> Spearman: {spearman_score:.4f}")
        print(f"   >> Pearson : {pearson_score:.4f}")
        
        results.append({
            "Model": model_name,
            "Spearman": spearman_score,
            "Pearson": pearson_score
        })
        
    except Exception as e:
        print(f"❌ Error: {e}")

# --- TABEL FINAL ---
print("\n" + "="*50)
print("📊 TABEL PERBANDINGAN PERFORMA MODEL")
print("="*50)
df_res = pd.DataFrame(results)
# Urutkan berdasarkan Spearman (karena ini metrik utama skripsi kamu)
df_res = df_res.sort_values(by="Spearman", ascending=False).reset_index(drop=True)

print(df_res)
df_res.to_csv("evaluasi_lengkap_model.csv", index=False)

c:\Python\Python3.11.9\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Mulai Evaluasi Komprehensif...

🔹 Menguji: Indo-E5 Small...


Batches: 100%|██████████| 94/94 [00:06<00:00, 14.58it/s]


   >> Spearman: 0.8894
   >> Pearson : 0.9110

🔹 Menguji: SimCSE IndoBERT...


Batches: 100%|██████████| 94/94 [00:24<00:00,  3.79it/s]


   >> Spearman: 0.8958
   >> Pearson : 0.9133

🔹 Menguji: NusaBERT Base...


Batches: 100%|██████████| 94/94 [00:28<00:00,  3.35it/s]


   >> Spearman: 0.8983
   >> Pearson : 0.9172

🔹 Menguji: NusaBERT Large...


Batches: 100%|██████████| 94/94 [01:15<00:00,  1.24it/s]


   >> Spearman: 0.8796
   >> Pearson : 0.8996

📊 TABEL PERBANDINGAN PERFORMA MODEL
             Model  Spearman   Pearson
0    NusaBERT Base  0.898317  0.917196
1  SimCSE IndoBERT  0.895776  0.913335
2    Indo-E5 Small  0.889431  0.910976
3   NusaBERT Large  0.879556  0.899570


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# ======================= KONFIGURASI MODEL =======================
MODEL_PATHS = {
    "nusabert-base": r"D:\\Media\\Kuliah\\Skripsi\\ScripTI\\Versi 4\\Model\\hf_models\\LazarusNLP__all-nusabert-base-v4",
    "sbert-nusabert-base": r"D:\\Media\\Kuliah\\Skripsi\\ScripTI\\Versi 4\\Model\\Trained_SBERT\\finetuned_all-nusabert-base_v1",
}


# ======================= PASANGAN KALIMAT UJI =======================
TEST_PAIRS = [
    # ---------- A: In-domain kuat ----------
    {
        "id": "A1",
        "group": "in_domain_strong",
        "desc": "Judul batik kawung, metode sama",
        "s1": "Pengenalan pola batik kawung menggunakan metode ellipse hough transform.",
        "s2": "Pengenalan motif batik kawung dengan pendekatan ellipse hough transform.",
    },
    {
        "id": "A2",
        "group": "in_domain_strong",
        "desc": "YOLOv5 deteksi ekspresi wajah",
        "s1": "Penelitian ini menggunakan YOLOv5 untuk mendeteksi ekspresi wajah secara real time.",
        "s2": "YOLOv5 digunakan dalam penelitian ini untuk deteksi ekspresi wajah secara waktu nyata.",
    },
    {
        "id": "A3",
        "group": "in_domain_strong",
        "desc": "Sistem informasi akademik",
        "s1": "Sistem informasi akademik ini dirancang untuk mengelola data mahasiswa, dosen, dan mata kuliah.",
        "s2": "Aplikasi informasi akademik dibangun untuk pengelolaan data mahasiswa, staf pengajar, serta mata kuliah.",
    },

    # ---------- B: In-domain sedang ----------
    {
        "id": "B1",
        "group": "in_domain_medium",
        "desc": "Klasifikasi batik vs daun (CNN sama, objek beda)",
        "s1": "Penelitian ini berfokus pada klasifikasi gambar motif batik menggunakan CNN.",
        "s2": "Penelitian ini berfokus pada klasifikasi citra daun menggunakan jaringan syaraf konvolusional.",
    },
    {
        "id": "B2",
        "group": "in_domain_medium",
        "desc": "Rekomendasi buku vs film",
        "s1": "Sistem ini memberikan rekomendasi buku berdasarkan riwayat peminjaman pengguna.",
        "s2": "Aplikasi ini menyarankan film berdasarkan rating yang diberikan pengguna sebelumnya.",
    },

    # ---------- C: Negasi ----------
    {
        "id": "C1",
        "group": "negation",
        "desc": "Menggunakan vs TIDAK menggunakan YOLO",
        "s1": "Penelitian ini menggunakan YOLOv5 sebagai metode utama deteksi objek.",
        "s2": "Penelitian ini tidak menggunakan YOLOv5 sebagai metode utama deteksi objek.",
    },
    {
        "id": "C2",
        "group": "negation",
        "desc": "Tujuan sistem vs TIDAK bertujuan",
        "s1": "Sistem ini bertujuan meningkatkan efisiensi proses pengolahan data transaksi.",
        "s2": "Sistem ini tidak bertujuan meningkatkan efisiensi proses pengolahan data transaksi.",
    },

    # ---------- D: Out-of-domain ----------
    {
        "id": "D1",
        "group": "out_of_domain",
        "desc": "Kalimat sehari-hari vs kalimat skripsi",
        "s1": "Hari ini saya pergi ke pasar untuk membeli sayur dan buah.",
        "s2": "Penelitian ini mengembangkan sistem deteksi kemiripan proposal tugas akhir.",
    },
    {
        "id": "D2",
        "group": "out_of_domain",
        "desc": "Hobi vs CNN",
        "s1": "Aku suka mendengarkan musik sambil bermain game di akhir pekan.",
        "s2": "Model CNN dilatih menggunakan dataset citra berukuran 224 kali 224 piksel.",
    },

    # ---------- E: Kalimat sama / hampir sama ----------
    {
        "id": "E1",
        "group": "identical",
        "desc": "Kalimat sama persis",
        "s1": "Dataset yang terbatas menjadi tantangan tersendiri dalam membangun model ini.",
        "s2": "Dataset yang terbatas menjadi tantangan tersendiri dalam membangun model ini.",
    },
    {
        "id": "E2",
        "group": "paraphrase_close",
        "desc": "Kalimat hampir sama, parafrase ringan",
        "s1": "Dataset yang terbatas menjadi tantangan tersendiri dalam membangun model ini.",
        "s2": "Keterbatasan dataset menjadi tantangan utama dalam membangun model ini.",
    },
]

# ======================= FUNGSI BANTU =======================

def evaluate_model(model_name: str, model: SentenceTransformer):
    rows = []
    for pair in TEST_PAIRS:
        s1 = pair["s1"]
        s2 = pair["s2"]

        emb1 = model.encode(s1, convert_to_tensor=True, normalize_embeddings=True)
        emb2 = model.encode(s2, convert_to_tensor=True, normalize_embeddings=True)

        sim = util.cos_sim(emb1, emb2).item()  # -1..1
        sim_clamped = max(0.0, min(1.0, sim))
        sim_percent = round(sim_clamped * 100, 1)

        rows.append({
            "model": model_name,
            "pair_id": pair["id"],
            "group": pair["group"],
            "desc": pair["desc"],
            "s1": s1,
            "s2": s2,
            "similarity_raw": sim,
            "similarity_percent": sim_percent,
        })
    return rows


def expected_text_for_group(group: str) -> str:
    if group == "in_domain_strong":
        return "Tinggi (mendekati 100%, karena topik & formulasi sangat mirip)."
    if group == "in_domain_medium":
        return "Sedang (masih cukup tinggi, tapi tidak setinggi in-domain kuat)."
    if group == "negation":
        return "Rendah (kalimat berlawanan makna, seharusnya tidak dianggap mirip)."
    if group == "out_of_domain":
        return "Sangat rendah (kalimat berbeda domain/topik)."
    if group == "identical":
        return "Sangat tinggi ≈ 100% (kalimat identik)."
    if group == "paraphrase_close":
        return "Tinggi (80–100%, karena parafrase dekat)."
    return "-"


# ======================= MAIN =======================

def main():
    if not MODEL_PATHS:
        print("Harap isi dulu MODEL_PATHS dengan path model yang ingin dibandingkan.")
        return

    all_rows = []

    for name, path in MODEL_PATHS.items():
        print(f"\n=== Load model: {name} ===")
        print(f"Path: {path}")
        model = SentenceTransformer(path)
        rows = evaluate_model(name, model)
        all_rows.extend(rows)

    df = pd.DataFrame(all_rows)

    # Simpan ke CSV tetap ada
    out_path = "hasil_perbandingan_model.csv"
    df.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"\nHasil detail disimpan ke: {out_path}")

    # ================= CETAK VERSI 'NARATIF' PER PASANGAN =================
    print("\n================ RINGKASAN PER PASANGAN KALIMAT ================\n")

    # Urutan model untuk label "Model A/B/C"
    model_names = list(MODEL_PATHS.keys())
    label_for_model = {name: f"Model {chr(ord('A') + i)}" for i, name in enumerate(model_names)}

    for pair in TEST_PAIRS:
        pid = pair["id"]
        group = pair["group"]
        desc = pair["desc"]

        print("=" * 60)
        print(f"Pair {pid} ({group} - {desc})")
        print("-" * 60)
        print(f"Kalimat A : {pair['s1']}")
        print(f"Kalimat B : {pair['s2']}\n")

        # hasil tiap model
        for name in model_names:
            label = label_for_model[name]
            row = df[(df["pair_id"] == pid) & (df["model"] == name)]
            if row.empty:
                continue
            percent = row.iloc[0]["similarity_percent"]
            print(f"Hasil {label} ({name:>25}) : {percent:5.1f}%")

        print(f"\nHarusnya (ekspektasi) : {expected_text_for_group(group)}\n")

    print("=" * 60)
    print("Selesai.\n")


if __name__ == "__main__":
    main()



=== Load model: nusabert-base ===
Path: D:\\Media\\Kuliah\\Skripsi\\ScripTI\\Versi 4\\Model\\hf_models\\LazarusNLP__all-nusabert-base-v4

=== Load model: sbert-nusabert-base ===
Path: D:\\Media\\Kuliah\\Skripsi\\ScripTI\\Versi 4\\Model\\Trained_SBERT\\finetuned_all-nusabert-base_v1

Hasil detail disimpan ke: hasil_perbandingan_model.csv

================ RINGKASAN PER PASANGAN KALIMAT ================

Pair A1 (in_domain_strong - Judul batik kawung, metode sama)
------------------------------------------------------------
Kalimat A : Pengenalan pola batik kawung menggunakan metode ellipse hough transform.
Kalimat B : Pengenalan motif batik kawung dengan pendekatan ellipse hough transform.

Hasil Model A (            nusabert-base) :  92.7%
Hasil Model B (      sbert-nusabert-base) :  98.1%

Harusnya (ekspektasi) : Tinggi (mendekati 100%, karena topik & formulasi sangat mirip).

Pair A2 (in_domain_strong - YOLOv5 deteksi ekspresi wajah)
-------------------------------------------------